###  Step 1: Train & Save the Model

train_model.py

In [ ]:
import joblib
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier

# Load data and train model
iris = load_iris()
X, y = iris.data, iris.target
model = RandomForestClassifier()
model.fit(X, y)

# Save model
joblib.dump(model, "iris_model.pkl")
print("Model saved as iris_model.pkl")


Run it locally or on your training environment:

In [ ]:
python train_model.py


### Step 2: Create FastAPI App

main.py

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import joblib
import numpy as np

app = FastAPI()

# Load model at startup
model = joblib.load("iris_model.pkl")

class IrisInput(BaseModel):
    data: list  # expects [[5.1, 3.5, 1.4, 0.2]]

@app.post("/predict/")
def predict(iris: IrisInput):
    prediction = model.predict(np.array(iris.data))
    return {"prediction": prediction.tolist()}


### Step 3: Create Requirements File

In [ ]:
fastapi
uvicorn
scikit-learn
joblib
numpy


### Step 4: Dockerfile

In [ ]:
FROM python:3.9-slim

WORKDIR /app

COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

COPY iris_model.pkl .
COPY main.py .

CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"]


### Step 5: Build and Run Docker Locally

In [ ]:
docker build -t iris-fastapi-app .
docker run -p 8000:8000 iris-fastapi-app


Test with:

In [ ]:
curl -X POST http://localhost:8000/predict/ \
-H "Content-Type: application/json" \
-d '{"data": [[5.1, 3.5, 1.4, 0.2]]}'


### Step 6: Kubernetes Deployment

In [ ]:
apiVersion: apps/v1
kind: Deployment
metadata:
  name: iris-fastapi
spec:
  replicas: 1
  selector:
    matchLabels:
      app: iris-fastapi
  template:
    metadata:
      labels:
        app: iris-fastapi
    spec:
      containers:
        - name: iris-fastapi
          image: your-dockerhub-username/iris-fastapi-app:latest
          ports:
            - containerPort: 8000
---
apiVersion: v1
kind: Service
metadata:
  name: iris-fastapi-service
spec:
  selector:
    app: iris-fastapi
  ports:
    - protocol: TCP
      port: 80
      targetPort: 8000
  type: LoadBalancer


### Step 7: Push Docker Image

In [ ]:
docker tag iris-fastapi-app your-dockerhub-username/iris-fastapi-app:latest
docker push your-dockerhub-username/iris-fastapi-app:latest


### Step 8: Deploy to Kubernetes

In [ ]:
kubectl apply -f k8s-deployment.yaml


###  Step 9: Access the Model

Once deployed, access the service using:

External IP (from kubectl get svc iris-fastapi-service)

Or expose using Ingress

Test the prediction endpoint similarly using curl or Postman.